In [1]:
!pip install tensorflow opencv-python

   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.6/39.0 MB 4.0 MB/s eta 0:00:10
   --- ------------------------------------ 3.1/39.0 MB 5.1 MB/s eta 0:00:07
   ---- ----------------------------------- 4.7/39.0 MB 5.9 MB/s eta 0:00:06
   ------ --------------------------------- 6.6/39.0 MB 6.5 MB/s eta 0:00:05
   -------- ------------------------------- 8.4/39.0 MB 6.9 MB/s eta 0:00:05
   ----------- ---------------------------- 10.7/39.0 MB 7.5 MB/s eta 0:00:04
   ------------- -------------------------- 12.8/39.0 MB 7.7 MB/s eta 0:00:04
   --------------- ------------------------ 15.5/39.0 MB 8.3 MB/s eta 0:00:03
   ------------------ --------------------- 17.8/39.0 MB 8.6 MB/s eta 0:00:03
   -------------------- ------------------- 19.9/39.0 MB 8.7 MB/s eta 0:00:03
   -----


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os

# Set up dataset paths (modify to your dataset location)
train_dir = 'C:/Users/GENAIKOLGPUSR32/Downloads/archive (1)/Dataset'
#validation_dir = 'C:/Users/GENAIKOLGPUSR32/Downloads/archive (1)/validation/'
validation_dir = 'C:/Users/GENAIKOLGPUSR32/Downloads/archive (1)/Dataset'
# Define treatment recommendations
disease_treatments = {
    'disease_1': 'Apply fungicide X and water the plant properly.',
    'disease_2': 'Use pesticide Y and prune infected leaves.',
    'disease_3': 'Water less frequently, and ensure good soil drainage.',
    'healthy': 'No treatment required. Plant is healthy.'
}

# Function to load and preprocess images
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resizing to the model's expected input size
    img = np.expand_dims(img, axis=0)  # Adding batch dimension
    img = img / 255.0  # Normalize to [0, 1]
    return img# Set up ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Get number of classes from the training data generator
num_classes = len(train_generator.class_indices)

def build_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # Output layer adjusted
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# model = build_model()

model = build_model()
model.summary()
# Train the model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator
)





def predict_disease(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    
    # Get the index with the highest confidence
    predicted_class_index = np.argmax(prediction)
    
    # Map the class index to the corresponding disease
    class_labels = list(disease_treatments.keys())
    predicted_disease = class_labels[predicted_class_index]
    
    # Get treatment recommendations
    treatment = disease_treatments.get(predicted_disease, "No treatment available.")
    
    return predicted_disease, treatment

# Example usage:
image_path = 'C:/Users/GENAIKOLGPUSR32/Downloads/archive (1)/validation/Apple___healthy/f902a6a0-a38b-441d-bd7e-5deb16ed709c___RS_HL 7727.jpg'
disease, treatment = predict_disease(image_path)
print(f"Predicted Disease: {disease}")
print(f"Recommended Treatment: {treatment}")




# Save the trained model
model.save('plant_disease_model.h5')

# Load the model for inference later
loaded_model = tf.keras.models.load_model('plant_disease_model.h5')










Found 35725 images belonging to 23 classes.
Found 35725 images belonging to 23 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 23)                  │          23,575 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,709,463 (98.07 MB)

 Trainable params: 2,121,751 (8.09 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

C:\Users\GENAIKOLGPUSR32\Python_Automation\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  66/1117 ━━━━━━━━━━━━━━━━━━━━ 51:47 3s/step - accuracy: 0.0734 - loss: 3.2785

In [ ]:
plot_model(model, to_file='plant_disease_cnn_architecture.png', show_shapes=True, show_layer_names=True)
print("Model architecture saved as 'plant_disease_cnn_architecture.png'")